In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os
import time

import snapshot_obj
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

# Massive compact non-luminous halos

Here, we inspect the properties of the clump of subhalos in the $\Lambda$CDM simulation with $v_\text{max} \sim v_\text{1kpc} > 30 \text{km/s}$.

## Construct selection arrays
Read $v_\text{max}$ and $v_\text{1kpc}$ and construct mask arrays for the peculiar dark subhalos, and for all dark subhalos in the same mass range.

In [ ]:
sim_id = "V1_LR_fix"
snap = snapshot_obj.Snapshot(sim_id, 127)
max_point = snap.get_subhalos("Max_Vcirc", "Extended")
vmax = max_point[:,0] * units.cm.to(units.km)
v1kpc = snap.get_subhalos("V1kpc", "Extended") * units.cm.to(units.km)

In [ ]:
masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
    snap, (1,0), (2,0))
mask_dark = dataset_compute.split_luminous(snap)[1]
mask_pecul = np.logical_and(vmax > 35, np.abs(vmax-v1kpc)/vmax < 0.15)
mask_pecul = np.logical_and.reduce([mask_isol, mask_dark, mask_pecul])
mask_ref = np.logical_and(vmax > 30, vmax < 50)
mask_ref = np.logical_and.reduce([mask_isol, mask_dark, mask_ref])

Inspect the constituents of the peculiar subhalos:

In [ ]:
masstype = snap.get_subhalos("MassType") * units.g.to(units.Msun)
for m_arr in masstype[mask_pecul]:
    print(["{:.2E}".format(m) for m in m_arr])

## Plot the rotation curves

Let us first see how many there are of these peculiar halos, and compare that to the number of all halos in the mass range:

In [ ]:
print(np.sum(mask_pecul))
print(np.sum(mask_ref))
print(np.sum(np.logical_and(mask_pecul, mask_ref)))

We see that the rotation curves of the subhalos in the given mass range divide into two classes: very compact ones, which constitute the clump in the V1kpc plots, and more regular looking ones.

In [ ]:
rot_curves = snap.get_subhalos('Vcirc', group='Extended/RotationCurve/All')
sub_offset = snap.get_subhalos('SubOffset', group='Extended/RotationCurve/All')

fig, axes = plt.subplots()
axes.set_xlim(-2, 20)

idxs = np.arange(mask_pecul.size)[mask_ref]
for i,idx in enumerate(idxs):
    v_circ = rot_curves[sub_offset[idx]:sub_offset[idx+1],0] * units.cm.to(units.km)
    radii = rot_curves[sub_offset[idx]:sub_offset[idx+1],1] * units.cm.to(units.kpc)
    if i==0:
        axes.plot(radii, v_circ, c='black', label="reference")
    else:
        axes.plot(radii, v_circ, c='black')

idxs = np.arange(mask_pecul.size)[mask_pecul]
for i,idx in enumerate(idxs):
    v_circ = rot_curves[sub_offset[idx]:sub_offset[idx+1],0] * units.cm.to(units.km)
    radii = rot_curves[sub_offset[idx]:sub_offset[idx+1],1] * units.cm.to(units.kpc)
    if i==0:
        axes.plot(radii, v_circ, c='gray', label="peculiar compact")
    else:
        axes.plot(radii, v_circ, c='gray')
    
plt.legend()
plt.savefig("rotation_curves_peculiar_{}.png".format(sim_id), dpi=200)

Let us see, what is the constitution of the mass near the centre. We calculate the rotation curves with bins of three particles:

In [ ]:
pts = {'all': None, 'gas': [0], 'dm': [1]}
v_circ = {}
radii = {}

for key, pt in pts.items():
    if pt is None:
        v, r = dataset_compute.compute_rotation_curves(snap, n_soft=3)
    else:
        v, r = dataset_compute.compute_rotation_curves(snap, n_soft=3, part_type=pt)

    v_circ[key] = v[mask_pecul] * units.cm.to(units.km)
    radii[key] = r[mask_pecul] * units.cm.to(units.kpc)
   

In [ ]:
for pt in pts.keys():
    print("Particle type: {}\n".format(pt) + \
          "Number of particles in the peculiar subhalos within radii\n" +\
          "   1kpc   2kpc")
    for r in radii[pt]:
        print("  {:3d}    {:3d}".format(np.sum(r < 1), np.sum(r < 2)))
    print('')

In [ ]:
fig, axes = plt.subplots()

axes.set_xlim(-2, 20)

for i, (v, r) in enumerate(zip(v_circ['dm'], radii['dm'])):
    if i==0:
        axes.plot(r, v, c='black', label="dm")
    else:
        axes.plot(r, v, c='black')

for i, (v, r) in enumerate(zip(v_circ['gas'], radii['gas'])):
    if i==0:
        axes.plot(r, v, c='grey', label="gas")
    else:
        axes.plot(r, v, c='grey')
        
plt.legend()

## Further inspection

Let us see, how far from the LG centre the peculiar subhalos are located:

In [ ]:
LG = dataset_compute.compute_LG_centre(snap, (1,0), (2,0))
dist_to_lg = dataset_compute.distance_to_point(snap, LG) * units.cm.to(units.kpc)
print(dist_to_lg[mask_pecul])

## Plot gas density vs. temperature

In [ ]:
data_grouped = dataset_compute.group_particles_by_subhalo(snap, "Density", "Temperature", part_type=[0])

In [ ]:
fig, axes = plt.subplots()
axes.set_xscale('log')
axes.set_yscale('log')
axes.set_xlim(5*10**3, 10**5)
axes.set_ylim(10**-30, 10**-22)

density = np.array(data_grouped["Density"])[mask_pecul] 
temp = np.array(data_grouped["Temperature"])[mask_pecul]
for rho, t in zip(density, temp):
    axes.scatter(t, rho)